### 二. SELECT语句优化

#### 2.1 如何思考 SELECT 语句优化

SELECT语句的优化, 处于所有sql优化中的最高优先级; 如下的这些SELECT优化方法, 同样适用于 `CREATE TABLE...AS SELECT`, `INSERT INTO...SELECT` 和 `DELETE...WHERE`语法. 主要的查询优化手段有: 
1. `SELECT ... WHERE` 慢查询的加速   
  首先, 考虑是否能加索引( index ). 在 where 条件的 column 上建立索引, 可以加快计算, 过滤和结果遍历. 为了避免浪费磁盘空间, 索引个数要少量有效   
  index 的相关内容, 参考8.3.1, 8.8.1

2. Isolate and tune any part of the query, such as a function call, that takes excessive time. Depending on how the query is structured, a function could be called once for every row in the result set, or even once for every row in the table, greatly magnifying any inefficiency.


3. 最小化查询时的全表扫描 ( full table scans ). 尤其是大表的扫描

4. 对表进行定期的 `ANALYZE TABLE` 统计, 便于优化器构建高效的执行计划

5. 不同的存储引擎 (storage engine) 有自己独特的优化技巧. 参考8.5.6的innodb引擎优化, 8.6.1mylsam引擎优化

6. Innodb的表, 可以对只读事务优化. 参考8.5.3

7. 不要对 query 语句做让人难以理解的变形, 特别是优化器可以对某些语句自动变形的时候

8. 当 sql 语句性能不佳时, 使用  EXPLAIN 查看 plan , 以便在 where , join 条件中使用索引. (当你非常熟练后, 使用 EXPLAIN 查看执行计划应该是所有调优的第一步)

9. 调整 mysql cache 的相关参数, 来高效使用  InnoDB 的 `buffer pool`; MyISAM 的 `key cache`, 和 `MySQL query cache`. 这回加速 sql 语句第二次及以后重复查询的执行速度. 因为查询结果可以直接从内存中获取. 

10. 增加可扩展性 (Scalability). 有时即使 query 使用了 cache 内存执行的很快也需要继续对它优化, 来降低 query 所需的 cache 大小., 提高应用的可扩展性. 可扩展性意味着应用可以同时响应更多用户, 更大的请求.

11. 对锁进行处理 (locking). "锁"会影响同一时间使用同一张表的其它 session 对该表的访问


#### 2.2 WHERE 条件优化
以下例子中, WHERE 条件都出现在 SELECT 语句中, 实际上这些优化方法同样适用于在 DELETE 和 UPDATE 语句中的 WHERE. 你可能热衷于重构 query 的格式, 牺牲可读性来加速 query , 但其实很多时候优化器会自动执行sql的重构, 因此只要保证 query 语句的可读性即可:  

1. 算术运算 
	* 移除不必要的括号
	```sql
    ((a AND b) AND c OR (((a AND b) AND (c AND d))))
		-> (a AND b AND c) OR (a AND b AND c AND d)
	```
	* 常量折叠
	```sql
	(a<b AND b=c) AND a=5
		-> b>5 AND b=c AND a=5
	```
	* 删除常量条件
	```sql
    (b>=5 AND b=5) OR (b=6 AND 5=5) OR (b=7 AND 5=6)
		-> b=5 OR b=6
	```
2. index 上的常量表达式只会被计算一次
3. 在单表上执行不带 where 条件的 `coun(*)`, 可以直接从 MyISAM 的 table information , 或内存表中直接获取
4. 如果不使用  GROUP BY , 则 `HAVING` 条件会被合并到 `WHERE` 中	
5, join 表上的 where 条件可以让表跳过一些行
6. 相比于 query 中的其它表, 常量表会被第一个读取. 常量表包括: 
	* 空表或只有一行的表
	* WHERE 条件执行在主键, 或唯一索引 (`PRIMARY KEY` or a `UNIQUE index`) 上的表
```sql
SELECT * FROM t WHERE primary_key=1;
SELECT * FROM t1,t2
	 	WHERE t1.primary_key=1 AND t2.primary_key=t1.id;
```
7. 有关join操作: 
	* 最佳的 join 组合方式, 会在尝试所有可能性后被找到. 比如, 如果所有  ORDER BY 和 GROUP BY 的列都来自同一张表, 则这个表是 join 时的首选
	* 如果 `ORDER BY` 从句和 `GROUP BY` 的条件不同, 或是 `ORDER BY` 和 `GROUP BY` 的列不是 join 的首选表, 则临时表被创建
	* 使用 `SQL_SMALL_RESULT` 修饰符, 临时表会完全在内存中被创建
8. 使用 index 还是 table scan 进行查询, 通常优化器都会选择使用 index, 除非优化器发现 index 涉及的数据查过了全表数据的30%. 目前来讲, 已经不存在一个确定的百分比能让优化器决定使用 index 还是 table scan. 优化器如今要考虑更多因素:  table size(表大小), number of rows(行数), and I/O block size(IO块的大小)
9. In some cases, MySQL can read rows from the index without even consulting the data file. If all columns used from the index are numeric, only the index tree is used to resolve the query.
10. 每行在被输出前, 所有不匹配 having 字句的行都会被跳过  

	

#### 2.3 range 优化
range 访问用于, 当条件中的列使用操作符  `=`, `<>`, `>`, `>=`, `<`, `<=`, `IS NULL`, `<=>`, `BETWEEN`, `LIKE` 或 `IN()` 和常量值进行比较时. 
```sql
SELECT * FROM tbl_name
  WHERE key_column = 10;

SELECT * FROM tbl_name
  WHERE key_column BETWEEN 10 and 20;

SELECT * FROM tbl_name
  WHERE key_column IN (10,20,30);

SELECT * FROM tbl_name
  WHERE key_part1 = 10 AND key_part2 IN (10,20,30)
```
以下章节, 描述优化器使用 range 访问时的条件
	